## 모델 저장과 복원
--- 
- 방법 1) 모델 전체 저장(모델 구조 + 가중치 + 변수)
    * save_model()
    * 복원시 추가 작업 필요 없이 모델 파일 로딩해서 사용 가능  

- 방법 2) 가중치만 저장
    * save_weights()
    * 복원 시 모델 구조(Achitechture) 생성 후 가중치 적용  

- 방법 3) 모델 전체 또는 가중치 자동 저장
    * fit()에서 ModelCheckPoint Event에 대한 callback 등록
    * save_best_only = True : 모니터링 기준에 따라서 좋은 성능의 모델만 저장
    * save_weight_only = True : 가중치만 저장

- 파일 또는 폴더
    * 파일 확장자가 없으면 폴더로 저장
    * 파일 확장자
        - **h5 / hdf5** : HDF5 포멧으로 모델 또는 가중치 저장
        - ckpf : 체크파일형태로 저장
        - pd : 모델 저장

### [1] 데이터 준비

In [1]:
from sklearn.datasets import load_iris
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import set_random_seed, plot_model
import numpy as np

In [2]:
set_random_seed(11)

In [3]:
iris = load_iris()

x_train = iris.data
y_train = iris.target

print(x_train.shape, y_train.shape)

(150, 4) (150,)


In [4]:
iris.feature_names, iris.target_names

(['sepal length (cm)',
  'sepal width (cm)',
  'petal length (cm)',
  'petal width (cm)'],
 array(['setosa', 'versicolor', 'virginica'], dtype='<U10'))

In [5]:
# petal length와 petal width와 만용
x_train = x_train[:, 2:]

In [6]:
# target : 문자 => 수치화
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

lencoder = LabelEncoder()
y_train = lencoder.fit(y_train).transform(y_train)

In [7]:
ohencoder = OneHotEncoder()
y_train = ohencoder.fit(y_train.reshape(-1, 1)).transform(y_train.reshape(-1, 1))

In [8]:
y_train=y_train.toarray()

In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(x_train)

x_train = scaler.transform(x_train)

In [10]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x_train, y_train
)

In [11]:
x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train
)

In [12]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)
print(x_val.shape, y_val.shape)


(84, 2) (84, 3)
(38, 2) (38, 3)
(28, 2) (28, 3)


### [3] 모델 구성 및 생성
- 입력 데이터 : 2개의 Feature
- 출력 데이터 : 3개 품종
- 학습 방법 : 분류 - 다중분류

In [13]:
irisModel = Sequential(name='irisModel')

irisModel.add(Dense(9, activation = 'relu', input_shape = (2,)))
irisModel.add(Dense(3, activation = 'softmax'))

In [14]:
irisModel.compile(loss = 'categorical_crossentropy',
optimizer = 'adam',
metrics = 'accuracy')

### [4] 모델 학습

In [15]:
irisModel.fit(x_train,y_train, epochs = 300 ,validation_data = (x_test, y_test))

Epoch 1/300
3/3 [==============================] - 1s 103ms/step - loss: 1.1961 - accuracy: 0.2619 - val_loss: 1.1538 - val_accuracy: 0.3947
Epoch 2/300
3/3 [==============================] - 0s 10ms/step - loss: 1.1822 - accuracy: 0.2619 - val_loss: 1.1393 - val_accuracy: 0.3947
Epoch 3/300
3/3 [==============================] - 0s 10ms/step - loss: 1.1682 - accuracy: 0.2619 - val_loss: 1.1249 - val_accuracy: 0.3947
Epoch 4/300
3/3 [==============================] - 0s 10ms/step - loss: 1.1545 - accuracy: 0.2619 - val_loss: 1.1108 - val_accuracy: 0.3947
Epoch 5/300
3/3 [==============================] - 0s 11ms/step - loss: 1.1406 - accuracy: 0.2619 - val_loss: 1.0971 - val_accuracy: 0.3947
Epoch 6/300
3/3 [==============================] - 0s 10ms/step - loss: 1.1279 - accuracy: 0.2619 - val_loss: 1.0835 - val_accuracy: 0.3947
Epoch 7/300
3/3 [==============================] - 0s 10ms/step - loss: 1.1145 - accuracy: 0.2738 - val_loss: 1.0703 - val_accuracy: 0.3947
Epoch 8/300
3/3 [==

### [5] 평가

In [16]:
lossv, accv  = irisModel.evaluate(x_val, y_val)

1/1 [==============================] - 0s 18ms/step - loss: 0.2043 - accuracy: 0.9286


In [17]:
M_File = 'irisModel.h5'
W_File = 'irisWeight.h5'

In [18]:
from tensorflow.keras.models import save_model

if accv >= 0.92:
    save_model(irisModel, M_File)
    irisModel.save_weights(W_File)

### [6] 복원
- 모델 또는 가중치 복원
- load_model(모델 파일 또는 모델 폴더명)
- Sequentail.load_weights(가중치 파일 또는 폴더명)

#### [6] - 1 모델 복원

In [19]:
from tensorflow.keras.models import load_model

In [20]:
reModel = load_model(M_File)
reModel.evaluate(x_test, y_test)

2/2 [==============================] - 0s 2ms/step - loss: 0.1919 - accuracy: 0.9211


[0.19194133579730988, 0.9210526347160339]

In [21]:
newModel = Sequential(name ='newIrisModel')
newModel.add(Dense(9, activation='relu', input_shape=(2,)))
newModel.add(Dense(3, activation='softmax'))

In [22]:
newModel.compile(loss = 'cattegorical_crossentropy',
optimizer='adam',
metrics='accuracy')

In [27]:
# 가중치 로딩
newModel.load_weights(W_File)

In [31]:
x_test

array([[ 0.99010798,  1.18556721],
       [ 0.8195957 ,  1.44883158],
       [ 0.76275827,  0.3957741 ],
       [ 0.53540856,  0.52740629],
       [ 0.87643312,  0.92230284],
       [-1.2833891 , -1.44707648],
       [ 0.36489628,  0.26414192],
       [-1.2833891 , -1.05217993],
       [-1.22655167, -1.3154443 ],
       [ 0.53540856,  0.26414192],
       [ 0.70592084,  0.3957741 ],
       [ 0.13754657, -0.26238682],
       [ 0.42173371,  0.3957741 ],
       [ 0.70592084,  1.05393502],
       [ 0.93327055,  1.18556721],
       [-1.39706395, -1.18381211],
       [ 1.0469454 ,  0.79067065],
       [ 0.42173371,  0.3957741 ],
       [ 0.76275827,  1.05393502],
       [ 0.76275827,  0.92230284],
       [ 0.08070915, -0.13075464],
       [-1.39706395, -1.3154443 ],
       [-1.56757623, -1.3154443 ],
       [ 0.42173371,  0.3957741 ],
       [-1.34022653, -1.44707648],
       [-1.22655167, -1.3154443 ],
       [ 0.76275827,  0.79067065],
       [ 0.36489628,  0.26414192],
       [-1.39706395,

In [33]:
newModel.weights

[<tf.Variable 'dense_2/kernel:0' shape=(2, 9) dtype=float32, numpy=
 array([[ 0.00528912,  0.36575663,  0.11749702,  1.0945088 ,  0.8929674 ,
         -1.0575484 ,  0.10247487, -0.6921785 , -0.31179088],
        [ 1.1772865 , -0.16279574,  0.89557385,  0.45831624, -0.3400395 ,
         -0.4681211 ,  0.54210734,  0.0621306 , -0.40548927]],
       dtype=float32)>,
 <tf.Variable 'dense_2/bias:0' shape=(9,) dtype=float32, numpy=
 array([-0.15883994, -0.1605565 , -0.09366003, -0.2953103 ,  0.74968874,
         0.21989846,  0.6463935 ,  0.64811987, -0.15751821], dtype=float32)>,
 <tf.Variable 'dense_3/kernel:0' shape=(9, 3) dtype=float32, numpy=
 array([[-0.17384325, -1.3666328 ,  0.02458954],
        [ 0.17922167, -0.743446  , -0.22730607],
        [-1.0335743 , -0.77647114,  1.264704  ],
        [-0.26151693, -0.4262719 ,  1.2653775 ],
        [-0.7579388 ,  1.0494908 , -0.3241362 ],
        [ 1.1942703 , -0.72513825, -0.10552017],
        [-1.1559618 ,  0.96711546,  0.56179476],
        [

In [30]:
newModel.evaluate(x_test, y_test)

ValueError: in user code:

    File "c:\Users\LJH\Anaconda3\lib\site-packages\keras\engine\training.py", line 1557, in test_function  *
        return step_function(self, iterator)
    File "c:\Users\LJH\Anaconda3\lib\site-packages\keras\engine\training.py", line 1546, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\LJH\Anaconda3\lib\site-packages\keras\engine\training.py", line 1535, in run_step  **
        outputs = model.test_step(data)
    File "c:\Users\LJH\Anaconda3\lib\site-packages\keras\engine\training.py", line 1501, in test_step
        self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\LJH\Anaconda3\lib\site-packages\keras\engine\training.py", line 948, in compute_loss
        return self.compiled_loss(
    File "c:\Users\LJH\Anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 184, in __call__
        self.build(y_pred)
    File "c:\Users\LJH\Anaconda3\lib\site-packages\keras\engine\